In [1]:
# class SimulationManager:
#     def __init__(self, config):
#         self.C = config
#         self.objects = []
#         self.previous_target = None

#     def setScenario(self, number_of_obstacles):
#         for i in range(number_of_obstacles):
#             # Add obstacle frames
#             parent_frame = 'parking_space_' + str(np.random.randint(1, 42))
#             if self.C.getFrame(parent_frame) is None:
#                 print(f"Error: Parent frame '{parent_frame}' does not exist.")
#                 continue

#             f = self.C.addFrame(f'car{i}', parent_frame)
#             f.setRelativePose('t(0 0 0.14) d(0 0 0 0)')
#             f.setShape(ry.ST.box, [0.9, 0.39, 0.28])
#             f.setColor([0.2, 0.2, 0.2, 0.4])
#             f.setContact(1)
#             f.setMass(1.0)

#     def getAttributes(self, object_name):
#         # Determine object type
#         if object_name.startswith('l_robot_base'):
#             obj_type = 'robot'
#         elif object_name.startswith('target_'):
#             obj_type = 'target'
#         elif object_name.startswith('car'):
#             obj_type = 'obstacle'
#         else:
#             obj_type = 'unknown'
  
#         # Retrieve the frame
#         frame = self.C.getFrame(object_name)
#         if frame is None:
#             print(f"Error: Frame for object '{object_name}' not found.")
#             return None

#         # Check if the object already exists
#         for obj in self.objects:
#             if obj['name'] == object_name:
#                 return obj

#         # Collect attributes and add to the list
#         attributes = {
#             "name": object_name,
#             "type": obj_type,
#             "frame": frame,
#             "position": frame.getPosition(),
#             "size": frame.getSize(),
#             "quaternion": frame.getQuaternion()
#         }
#         self.objects.append(attributes)
#         return attributes

#     def setTarget(self, object_name):
#         # Retrieve the frame
#         frame = self.C.getFrame(object_name)
#         if frame is None:
#             print(f"Error: Frame for object '{object_name}' not found.")
#             return None

#         # Revert previous target's color
#         if self.previous_target:
#             prev_frame = self.C.getFrame(self.previous_target)
#             if prev_frame:
#                 prev_frame.setColor([0.2, 0.2, 0.2, 0.4])

#         # Update current target's color
#         frame.setColor([0, 1, 0, 0.4])

#         # Update object type in the list
#         for obj in self.objects:
#             if obj['name'] == self.previous_target:
#                 obj['type'] = 'obstacle'
#             if obj['name'] == object_name:
#                 obj['type'] = 'target'

#         # Store the current target
#         self.previous_target = object_name

# # Initialize simulation
# C = ry.Config()
# C.addFile('/home/alperen/Desktop/cs449_project/scenarios/simEnv.g')

# # Create a SimulationManager instance
# sim_manager = SimulationManager(C)

# # Retrieve attributes for all objects
# for frame_name in C.getFrameNames():
#     sim_manager.getAttributes(frame_name)

# # Set an object as the target
# sim_manager.setTarget('car2')

# # Retrieve attributes for the target
# g = sim_manager.objects
# # View the simulation
# C.view()


In [2]:
# # Set up the scenario
# sim_manager.setScenario(5)

# # Retrieve attributes for the target
# g = sim_manager.objects
# # View the simulation
# C.view()

In [1]:
import robotic as ry
import numpy as np
import time

C = ry.Config() 

In [2]:
class Robot:
    def __init__(self, name, baseFrame, middleFrame):
        self.name = name
        self.baseFrame = baseFrame       # e.g. "robot1_base"
        self.middleFrame = middleFrame   # e.g. "robot1_middle_joint"
        self.currentQ = None             # Stores the last known joint state
        self.busy = False                # Indicates if the robot is currently executing a task

    def setJointState(self, C, q):
        """ Helper method to set joint state in the configuration and update local state. """
        C.setJointState(q)
        self.currentQ = q

In [3]:
class RobotManager:
    def __init__(self, config, robots):
        """
        Initialize the RobotManager with a configuration and list of Robot objects.

        Parameters:
        - config: The ry.Config instance for the robotic environment.
        - robots: List of Robot objects representing the robots in the environment.
        """
        self.C = config
        self.robots = robots

    def findNearestAvailableRobot(self, locationFrame):
        """
        Find the nearest available robot to a given location frame.

        Parameters:
        - locationFrame: Name of the frame representing the location.

        Returns:
        - Nearest available Robot object or None if no free robots are available.
        """
        free_robots = [r for r in self.robots if not r.busy]
        if not free_robots:
            print("No free robots available!")
            return None

        # Calculate distance from each free robot's base frame to the location frame
        min_dist = float('inf')
        chosen_robot = None
        for robot in free_robots:
            robot_pos = C.getFrame(robot.baseFrame).getPosition()
            loc_pos = C.getFrame(locationFrame).getPosition()
            dist = np.linalg.norm(np.array(robot_pos) - np.array(loc_pos))
            if dist < min_dist:
                min_dist = dist
                chosen_robot = robot
        return chosen_robot

    def dispatchMoveToTarget(self, robot, targetFrame):
        """
        Dispatch a robot to move to a target frame.

        Parameters:
        - robot: Robot object to dispatch.
        - targetFrame: Name of the frame representing the target location.

        Returns:
        - True if the dispatch was successful, False otherwise.
        """
        qStart = robot.currentQ
        robot.busy = True
        # Get inverse kinematics solution to move to the target frame
        qIK = self.moveToTarget(
            robot.baseFrame,
            robot.middleFrame,
            targetFrame
        )

        if qIK is None:
            print(f"Failed to find a feasible path for {robot.name} to {targetFrame} in moveCar")
            robot.busy = False
            return False

        # Plan path using RRT or similar path planner
        success = self.planAndExecutePath(robot, qStart, qIK)
        robot.busy = False
        robot.currentQ = qIK
        return success

    def dispatchMoveToPark(self, robot, targetFrame, carFrame):
        """
        Dispatch a robot to move to a target frame.

        Parameters:
        - robot: Robot object to dispatch.
        - targetFrame: Name of the frame representing the target location.

        Returns:
        - True if the dispatch was successful, False otherwise.
        """
        qStart = robot.currentQ
        robot.busy = True
        # Get inverse kinematics solution to move to the target frame
        qIK = self.moveToPark(
            robot.middleFrame,
            targetFrame,
            carFrame
        )

        if qIK is None:
            print(f"Failed to find a feasible path for {robot.name} to {targetFrame} in moveToPark")
            robot.busy = False
            return False

        # Plan path using RRT or similar path planner

        success = self.planAndExecutePath(robot, qStart, qIK)
        robot.busy = False
        return success

    def moveToTarget(self, baseFrame, middleFrame, targetFrame):
        """
        Compute inverse kinematics to move the robot to a target frame.

        Parameters:
        - baseFrame: Name of the base frame of the robot.
        - middleFrame: Name of the middle joint frame of the robot.
        - targetFrame: Name of the target frame to move towards.

        Returns:
        - Joint configuration (q) if a feasible path is found, None otherwise.
        """
        q0 = self.C.getJointState()
        komo = ry.KOMO(self.C, 5, 10, 1, False)
        komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq, [1e-1])
        komo.addObjective([], ry.FS.jointState, [], ry.OT.sos, [1e-1], q0)
        komo.addObjective([], ry.FS.positionDiff, [middleFrame, targetFrame], ry.OT.eq, [1e1], [0, -0.1, 0])
        komo.addObjective([], ry.FS.scalarProductXX, [baseFrame, targetFrame], ry.OT.eq, [1e2], [-1])
   
        ret = ry.NLP_Solver(komo.nlp(), verbose=0).solve()
        if ret:
            q = komo.getPath()[-1]
            return q
        else:
            return None
        
    def moveToPark(self, baseFrame, targetFrame, carFrame):
            q0 = self.C.getJointState()
            if targetFrame.getPosition()[0] > 0:
                #x = -0.6
                product = 1
            else: 
                #x = 0.6
                product = -1

            komo = ry.KOMO(C, 5, 10, 1, False)
            komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq, [1e-1])
            komo.addObjective([], ry.FS.jointState, [], ry.OT.sos, [1e-1], q0)
            komo.addObjective([], ry.FS.positionDiff, [carFrame, targetFrame.name], ry.OT.eq, [1e0], [0, -0.1, 0.15])
            komo.addObjective([], ry.FS.scalarProductXX, [baseFrame, targetFrame.name], ry.OT.eq, [1e2], [product])
            ret = ry.NLP_Solver(komo.nlp(), verbose=0 ).solve()
            q = komo.getPath()[-1]

            return q

    def planAndExecutePath(self, robot, qStart, qIK):
        """
        Plan and execute a path for the robot to the given joint configuration.

        Parameters:
        - robot: Robot object to plan path for.
        - qIK: Target joint configuration (from inverse kinematics).

        Returns:
        - True if path planning and execution were successful, False otherwise.
        """
        rrt = ry.PathFinder()
        start = robot.currentQ if robot.currentQ is not None else self.C.getJointState()
        rrt.setProblem(self.C, [qStart], [qIK])

        feasible_path = None
        for trial in range(10):
            ret = rrt.solve()
            if ret.feasible:
                feasible_path = ret.x
                break

        if feasible_path is None:
            print(f"Failed to find a feasible path for {robot.name} to qIK")
            return False

        # Execute the path
        for t in range(feasible_path.shape[0]):
            self.C.setJointState(feasible_path[t])
            robot.currentQ = feasible_path[t]
            self.C.view()
            time.sleep(0.1)

        return True


In [5]:
# Initialize your configuration and create Robot objects
C = ry.Config()
C.addFile("simEnv2.g")

robot1 = Robot(name="R1", baseFrame="l_robot_base", middleFrame="l_middle_joint")
robot1.currentQ = C.getJointState()

# Create RobotManager with the configuration and robots
manager = RobotManager(C, [robot1])

# Example scenario: A new car arrives at a drop-off point
def newCarArrived(carFrameName):
    carFrameName = "car1"
    parkingFrame = C.getFrame("parking_space_20")

    # Find the nearest available robot to the drop-off point
    chosen_robot = manager.findNearestAvailableRobot(carFrameName)
    if chosen_robot:
        # Dispatch the robot to move to the drop-off point
        success = manager.dispatchMoveToTarget(chosen_robot, carFrameName)
        if success:
            # Attach the car to the robot
            C.getFrame(carFrameName).unLink()
            C.attach(chosen_robot.baseFrame, carFrameName)

            # Move the robot to the parking space
            success = manager.dispatchMoveToPark(chosen_robot, parkingFrame, carFrameName)
            # if success:
            #     # Detach the car from the robot
            #     C.getFrame(carFrameName).unLink()

            #     # Return the robot to its initial position or another designated position
            #     manager.dispatchMoveToTarget(chosen_robot, "initial_position")

    else:
        print("No available robot to handle the new car.")

newCarArrived("car1")


-- RRT_PathFinder.cpp:RRT_PathFinder:258(0) initializing with infeasible qT:
query: h_goal: 0 g_coll: 0 isGoal: 1 isFeasible: 0
proxy:  (l_robot_body)-(car1) [97,49] 	d=-0
proxy:  (l_robot_base)-(car1) [96,49] 	d=-0
-- RRT_PathFinder.cpp:RRT_PathFinder:257(0) initializing with infeasible q0:
query: h_goal: 0 g_coll: 0 isGoal: 1 isFeasible: 0
proxy:  (l_robot_base)-(car1) [96,49Failed to find a feasible path for R1 to qIK
] 	d=-0
proxy:  (l_robot_body)-(car1) [97,49] 	d=-0
-- RRT_PathFinder.cpp:RRT_PathFinder:258(0) initializing with infeasible qT:
query: h_goal: 0 g_coll: 0 isGoal: 1 isFeasible: 0
proxy:  (l_robot_base)-(car1) [96,49] 	d=-0
proxy:  (l_robot_body)-(car1) [97,49] 	d=-0


In [ ]:
C.addFile("simEnv2.g")

C.view()

In [ ]:
print(C.getFrame('l_robot_base').getPosition())
print(C.getJointState())

In [4]:
def moveToTarget(C, target):
    q0 = C.getJointState()

    komo = ry.KOMO(C, 5, 10, 1, False)
    komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq, [1e-1])
    komo.addObjective([], ry.FS.jointState, [], ry.OT.sos, [1e-1], q0)
    komo.addObjective([], ry.FS.positionDiff, ['l_middle_joint', target.name], ry.OT.eq, [1e1], [0.1, -0.1, 0])
    komo.addObjective([], ry.FS.scalarProductXX, ['l_robot_base', target.name], ry.OT.eq, [1e2], [-1])
    ret = ry.NLP_Solver(komo.nlp(), verbose=0 ).solve()
    q = komo.getPath()[-1]

    return q

In [5]:
def moveToPark(C, target):
    q0 = C.getJointState()
    if target.getPosition()[0] > 0:
        x = -0.6
        product = 1
    else: 
        x = 0.6
        product = -1

    komo = ry.KOMO(C, 5, 10, 1, False)
    komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq, [1e-1])
    komo.addObjective([], ry.FS.jointState, [], ry.OT.sos, [1e-1], q0)
    komo.addObjective([], ry.FS.positionDiff, ['car1', target.name], ry.OT.eq, [1e0], [0, -0.1, 0.15])
    komo.addObjective([], ry.FS.scalarProductXX, ['l_robot_base', target.name], ry.OT.eq, [1e2], [product])
    ret = ry.NLP_Solver(komo.nlp(), verbose=0 ).solve()
    q = komo.getPath()[-1]

    return q

In [6]:
def moveBack(C):
    q0 = C.getJointState()
    
    komo = ry.KOMO(C, 5, 10, 1, False)
    komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq, [1e-1])
    komo.addObjective([], ry.FS.jointState, [], ry.OT.sos, [1e-1], q0)
    komo.addObjective([], ry.FS.positionDiff, ['l_middle_joint', 'robot_start'], ry.OT.eq, [1e1], [0, 0.05, 0])
    komo.addObjective([], ry.FS.scalarProductXX, ['l_middle_joint', 'robot_start'], ry.OT.eq, [1e2], [1])
    ret = ry.NLP_Solver(komo.nlp(), verbose=0 ).solve()
    q = komo.getPath()[-1]
    
    return q

In [7]:
car = C.getFrame('car1')
qHome = C.getJointState()
robot_to_car = moveToTarget(C, car)

In [ ]:
rrt = ry.PathFinder()
rrt.setProblem(C, [qHome], [robot_to_car])

for trial in range(10):
    ret = rrt.solve()
    path = ret.x
    print(ret.feasible)
    if ret.feasible is False:
        continue
    else:
        break

In [ ]:
C.view()
time.sleep(.8)
# display the path
for t in range(0, path.shape[0]-1):
    C.setJointState(path[t])
    C.view()
    time.sleep(.1)

In [10]:
#attach cargo
C.getFrame('car1').unLink()
C.attach('l_robot_base', 'car1') 


In [11]:
target = C.getFrame('parking_space_20')
#robotPos = C.getFrame('l_robot_base').getPosition()
robot_to_park = moveToPark(C, target)   

In [ ]:
rrt2 = ry.PathFinder()
rrt2.setProblem(C, [robot_to_car], [robot_to_park])

# ret2 = rrt2.solve()
# path2 = ret2.x
for trial in range(10):
    ret2 = rrt2.solve()
    path2 = ret2.x
    print(ret2.feasible)
    if ret2.feasible is False:
        continue
    else:
        break

In [13]:
C.view()
time.sleep(.8)
# display the path
for t in range(0, path2.shape[0]-1):
    C.setJointState(path2[t])
    C.view()
    time.sleep(.1)

In [ ]:
#detach cargo
C.getFrame('car1').unLink()

In [ ]:
return_back = moveBack(C)

rrt3 = ry.PathFinder()
rrt3.setProblem(C, [robot_to_park], [return_back])

ret3 = rrt3.solve()
path3 = ret3.x

In [16]:
C.view()
time.sleep(.8)
# display the path
for t in range(0, path3.shape[0]-1):
    C.setJointState(path3[t])
    C.view()
    time.sleep(.1)

In [ ]:
print(C.getFrame("car2").getPosition())
print(C.getFrame("parking_space_23").getPosition())

In [18]:
#C.setJointState(qHome)

# ry.params_add({'botsim/verbose': 2., 'physx/motorKp': 10000., 'physx/motorKd': 1000.})
# ry.params_print()
# bot = ry.BotOp(C, False)
# bot.home(C)

# bot.move(robot_to_car, [2. , 3.])
# while bot.getTimeToEnd()>0:
#     bot.sync(C, .1)

# bot.moveAutoTimed(path, 1., 1.)
# while bot.getTimeToEnd()>0:
#     bot.sync(C, .1)

# C.getFrame('car2').unLink()
# C.attach('l_robot_base', 'car2') 

# target = C.getFrame('parking_space_28')
# #robotPos = C.getFrame('l_robot_base').getPosition()
# robot_to_park = moveToPark(C, target)

# rrt2 = ry.PathFinder()
# rrt2.setProblem(C, [robot_to_car], [robot_to_park])

# for trial in range(10):
#     ret2 = rrt2.solve()
#     path2 = ret2.x
#     print(ret2.feasible)
#     if ret2.feasible is False:
#         continue
#     else:
#         break

# bot.moveAutoTimed(path2, 1., 1.)
# while bot.getTimeToEnd()>0:
#     bot.sync(C, .1)

# C.getFrame('car2').unLink()

# return_back = moveBack(C)

# rrt3 = ry.PathFinder()
# rrt3.setProblem(C, [robot_to_park], [return_back])

# ret3 = rrt3.solve()
# path3 = ret3.x

# bot.moveAutoTimed(path3, 1., 1.)
# while bot.getTimeToEnd()>0:
#     bot.sync(C, .1)


# for t in range(0, path.shape[0]-1):
#     bot.sync(C, .1)
#     bot.moveTo(path[t])

In [19]:
# del bot